
En este cuaderno, calcularemos la importancia de característica para diferentes metodos de ensemblaje usando árboles, para predecir los corrimientos al rojo fotométricos para 6 bandas fotométricas (u, g, r, i, z, y). Acompaña a Capítulo 6 del libro.

Autora: Viviana Acquaviva. Traducido por Lucia Perez y Rosario Cecilio-Flores-Elie. 

License: TBD

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)


font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
#matplotlib.rcParams.update({'figure.autolayout': True})
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

In [ ]:
sel_features = pd.read_csv('../data/sel_features.csv', sep = '\t')

In [ ]:
sel_target = pd.read_csv('../data/sel_target.csv')

In [ ]:
sel_features.shape

In [ ]:
sel_target.values.ravel() #changes shape to 1d row-like array

### Primeros serán los Bosques Aleatorios

In [ ]:
model = RandomForestRegressor(max_features=4, n_estimators=200) #Mosca de re-asignar el estado aleatorio

Después de encajar el modelo, tendrá el atributo de "feature\_importances\_" (importancias de característica). Así las podemos estudiar:

In [ ]:
model.fit(sel_features, sel_target.values.ravel()) 

# nota: esto no parte los datos entre entrenamiento/prueba, está encajando con el conjunto entero

In [ ]:
model.feature_importances_

Y este código marca las importancias de característica:

In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

# Emprimir los rangos de características:
print("Feature ranking:")

for f in range(sel_features.shape[1]):
    print("%d. feature: %s, %d (%f)" % (f + 1, sel_features.columns[indices[f]], indices[f], importances[indices[f]]))

# Marcar las importancias de características del bosque:
plt.figure(figsize=(16,6))
plt.title("Feature importances")
plt.bar(range(sel_features.shape[1]), importances[indices],
       color="r", align="center")
plt.xticks(range(sel_features.shape[1]), sel_features.columns[indices])
plt.xlim([-1, sel_features.shape[1]])
plt.show()

### En este ejemplo, todas las características son importantes, pero es difícil diagnosticar problemas.

### Lo que se puede hacer es comparar con los resultados de otros algoritmos.

In [ ]:
# Marcar las importancias de características de los tres métodos:

plt.figure(figsize=(16,6))

plt.title("Feature importances for various models")

models = [RandomForestRegressor(max_features=4, n_estimators=200), \
          AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=None), n_estimators=100), 
          xgb.XGBRegressor(objective ='reg:squarederror', max_depth=6, n_estimators = 500, learning_rate=0.1)]

model_names = ['Random Forests', 'AdaBoost', 'XGBoost']

for i, model in enumerate(models):

    model.fit(sel_features, sel_target.values.ravel()) 
    
    importances = model.feature_importances_
    
    indices = np.argsort(importances)[::-1]
    
    plt.bar(np.arange(sel_features.shape[1])+0.1*i, importances, 
            align="center", width=0.7, alpha = 0.5, label = model_names[i])
    
    plt.xticks(range(sel_features.shape[1]), sel_features.columns)
    
    plt.xlim([-1, sel_features.shape[1]])
    
    plt.legend(fontsize = 12)   
    
#    print('For model', model_names[i], 'features importances are', sel_features.columns[indices].values, importances[indices])

Tenemos que acordarnos que la importancia de característica es solo una señal, y casi siempre depende en cual algoritmo se usó. 

### Sumario del problema de fuga en las etiquetas de prueba

- Cuando optimizamos parámetros usando una búsqueda en cuadro, escogemos los parámetros que tienes las mejores notas de prueba. Esto es diferente a lo que pasaría con nuevos datos -- para ser justos, absolutamente no podemos mirar a las etiquetas de pruebas mientras estamos entrenando. Entonces, se tiene que usar la <b> validación cruzada anidada </b> para evaluar el error en generalizar, para no tener fuga entre el proceso de optimizar los parámeteros y el proceso de validación cruzada.
<br>

- Técnicamente, estandarizar o normalizar los datos usando el conjunto entere de aprendizaje crea fuga entre los conjuntos de entrenamiento y de prueba (el conjunto de entrenamiento 'sabe' del promedio y la desviación estándar de todos los datos). Esto usalmente no es tan dramático, pero lo correcto sería estandarizar o normalizar solo el conjunto de entrenamiento entre cada iteración de validación cruzada (ó: después de separar entre entrenamiento y prueba), y aplicar la misma transformación al conjunto de prueba. Así el modelo es un !!!canal!!!.
<br>

- Técnicamente, seleccionando características usando el conjunto entero de aprendizaje crea fuga entre los conjuntos de entrenamiento y de prueba (el modelo 'escoge' las características que dan los mejores resultados con el conjunto de prueba). Una forma de prevenir esto es escoger las características que son buenas pero no las mejores (las mediocres) entre un modelo con validación cruzada. !!!clarify meaning!!!A possible solution is to pick the "average" best features within a cross-validated model. 
</br>

- Alternativamente , podemos usar procesos sin supervisión, por ejemplo escogiendo las características con la divergencia más alta. Esto lo podemos hacer con todos el conjunto de aprendizaje entero, porque no usa etiquetas--pero no elegirá las características pertinentes a un problema específico con supervisión. 